# Dataset / Layer cloning template  
This notebook illustrates the steps needed to clone a dataset or a layer in GFW. The [Resource Watch API](https://api.resourcewatch.org/) (where the assets are stored) provides with a [cloning option](https://resource-watch.github.io/doc-api/reference.html#cloning-a-dataset) directly, but the downside is that it requires to clone each component (dataset, layers, metadata, vocabulary) independently, which makes the process cumbersone and much more error-prone.  
This notebook provides a template to clone a full dataset (with all its components) using the [**LMIpy library**](https://lmipy.readthedocs.io/en/latest/), which was built for operations with RW datasets and is therefore much more efficient and less error-prone.

In [1]:
# Importing the necessary libraries
import os
import json
from IPython.display import display
import getpass
import requests

In [2]:
# Install LMIpy if needed
#!pip install LMIPy
#!python setup.py develop 

from IPython.display import clear_output
clear_output()

import LMIPy as lmi
from LMIPy import utils, dataAPI

print(f'LMI ver. {lmi.__version__} ready!')

LMI ver. 0.6.2 ready!


## Authentiction  
To modify Resource Watch API's assets, the user needs a RW API account ([see here on creation](https://resource-watch.github.io/doc-api/quickstart.html#2-create-an-account-with-the-rw-api)) with editor privileges, and use it to retrieve the API token key.

In [ ]:
email = getpass.getpass('Login email:')
password = getpass.getpass('Login password:')

In [ ]:
payload = {
    "email": f"{email}",
    "password": f"{password}"
}

url = f'https://api.resourcewatch.org/auth/login'

headers = {'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)

API_TOKEN = r.json().get('data').get('token')

## Clone a dataset and release in production 
On this step, the new datset is cloned and moved from staging to production, while the old dataset is unpublished.

### 1) load staging and (current) production datasets (if not already) via id  

In [ ]:
ds = lmi.Dataset('')
ds

In [ ]:
ds_prod = lmi.Dataset('')
ds_prod

### 2) Clone staging dataset to production  
Clone dataset using the `clone` method from LMIpy. Update dataset asset name.  
Change environment to `production` afterwards (it does not work during clonning)

In [ ]:
dataset_name = 'SET NAME (v202205)'


ds_clone = ds.clone(
    token=API_TOKEN,
    env='production',
    dataset_params={
        'name': dataset_name,
        'application': ['gfw']
    },
    clone_children=True
)
#Environment does not update with clone, needs to be updated after
ds_clone.update(update_params={'env':'production'}, token=API_TOKEN) 

### 3) Unpublish old production dataset  
Instead of deleting the old dataset, it is suggested to unpublish it. This way, the dataset is not lost and can be recovered as a back-up if necessary.  
Update name to indicate old dataset.

In [ ]:
ds_prod.update(update_params={
    'name': 'SET NAME (DEPRECATED)',
    'published':False}, token=API_TOKEN)